In [2]:
#Basic setup
#Import main library
import torch
import torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

#Import common library
import numpy as np
import os, json, cv2, random

#Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
print(np.__version__)

1.26.4


In [4]:
#Load the model and the weights
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
#cfg.MODEL.DEVICE = "cuda"

predictor = DefaultPredictor(cfg)

[10/16 16:17:37 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl ...


In [5]:
print(torch.__version__)
print(torchvision.__version__)

2.0.0
0.15.0


In [6]:
img = cv2.imread("pose1.jpg")
print(img.shape[:2])

(1080, 1920)


In [7]:
img_resized = cv2.resize(img, (800, 450))
print(img_resized.shape[:2])

(450, 800)


In [8]:
predictions = predictor(img_resized)

/home/hatsu/miniconda3/envs/detectron/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
print(predictions)
print(predictions["instances"].pred_classes)
print(predictions["instances"].pred_boxes)

tensor([0], device='cuda:0')
Boxes(tensor([[245.9251,  14.2282, 566.4948, 443.6218]], device='cuda:0'))


In [11]:
viz = Visualizer(img_resized[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                 instance_mode=ColorMode.IMAGE)

output = viz.draw_instance_predictions(predictions["instances"].to("cpu"))

cv2.imshow("Result", output.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()